# Preprocessing

### External preprocessing
- Generates weights for each combination of postal area and gender (male, female, unknown)

Chloropleth map of proportion of 18-24 year olds by postcode

### Consumer preprocessing
- Set null postcode for postcodes that didnt match state

Chloropleth map of customers by postal area

### Merchant preprocessing
- Clean tags (25 unique tags)
- Separate tags into 5 segments

Load in 25 unique tags and the 5 segments

### Transaction preprocessing
- Remove transactions not between 5 and 10000
- Apply weights (include weighting formula)

Add some time series charts for a select few abns to justify use of forecasting model

# Modelling

### Pytorch neural network
- Ben will explain model

Evaluation charts

# Results
- Results by segment and total